In [2]:
# Import the dependencies.
import pandas as pd
import json
import pandas.io.json as pd_json
import censusdata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

import pickle
#import xgboost as xgb

# Import the requests library.
import requests

api_key = "24f7bba880a1af36816cec59796a7e4f07da5789"

In [3]:
# Import ACS Data Profile variables
dp_var_url = "https://api.census.gov/data/2018/acs/acs5/profile/variables.json"
dp_acs_vars = requests.get(dp_var_url).json()
dp_acs_vars = dp_acs_vars["variables"]

dp_vars_df = pd.DataFrame.from_dict(dp_acs_vars, orient = 'index')
dp_vars_df.index.name = 'code'

In [4]:
#Import ACS Detail variables
b_var_url = "https://api.census.gov/data/2018/acs/acs5/variables.json"
b_acs_vars = requests.get(b_var_url).json()
b_acs_vars = b_acs_vars["variables"]

b_vars_df = pd.DataFrame.from_dict(b_acs_vars, orient = 'index')
b_vars_df.index.name = 'code'

In [5]:
# URL for ACS Data Profile call
dp_var_list = ["DP02_0122E", "DP02_0001E", "DP02_0015E", "DP03_0001E", "DP03_0005PE",
               "DP04_0001E", "DP04_0050E"]

dp_url_start = "https://api.census.gov/data/2018/acs/acs5/profile?get=NAME"
dp_url_end =  "&for=county:*&in=state:*&key=" + api_key

dp_url = dp_url_start

for var in dp_var_list:
    dp_url = dp_url + ',' + var

dp_url = dp_url + dp_url_end

print(dp_url)

https://api.census.gov/data/2018/acs/acs5/profile?get=NAME,DP02_0122E,DP02_0001E,DP02_0015E,DP03_0001E,DP03_0005PE,DP04_0001E,DP04_0050E&for=county:*&in=state:*&key=24f7bba880a1af36816cec59796a7e4f07da5789


In [6]:
# URL for ACS Detail call
b_var_list = ["B01001_001E", "B01002_001E","B06001_013E","B06009_002E","B06009_003E","B06009_004E","B06009_005E","B06009_006E",
              "B06012_002E","B08133_001E","B19013_001E","B19301_001E","B19326_001E","B25071_001E",
              "B25077_001E"]

b_url_start = "https://api.census.gov/data/2018/acs/acs5?get=NAME"
b_url_end =  "&for=county:*&in=state:*&key=" + api_key

b_url = b_url_start

for var in b_var_list:
    b_url = b_url + ',' + var

b_url = b_url + b_url_end

print(b_url)

https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E,B01002_001E,B06001_013E,B06009_002E,B06009_003E,B06009_004E,B06009_005E,B06009_006E,B06012_002E,B08133_001E,B19013_001E,B19301_001E,B19326_001E,B25071_001E,B25077_001E&for=county:*&in=state:*&key=24f7bba880a1af36816cec59796a7e4f07da5789


In [7]:
#Print variable code names
dp_codes = []
b_codes = []

for var in dp_var_list:
    dp_codes.append(dp_vars_df.loc[var, 'label'])

for var in b_var_list:
    b_codes.append(b_vars_df.loc[var, 'label'])
    
print(dp_codes, b_codes)

['Estimate!!ANCESTRY!!Total population', 'Estimate!!HOUSEHOLDS BY TYPE!!Total households', 'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Average household size', 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over', 'Percent Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed', 'Estimate!!HOUSING OCCUPANCY!!Total housing units', 'Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units'] ['Estimate!!Total', 'Estimate!!Median age --!!Total', 'Estimate!!Total!!Born in state of residence', 'Estimate!!Total!!Less than high school graduate', 'Estimate!!Total!!High school graduate (includes equivalency)', "Estimate!!Total!!Some college or associate's degree", "Estimate!!Total!!Bachelor's degree", 'Estimate!!Total!!Graduate or professional degree', 'Estimate!!Total!!Below 100 percent of the poverty level', 'Estimate!!Aggregate travel time to work (in minutes)', 'Estimate!!Median household income in the past 12 months 

In [8]:
codes_dict = {'code' : dp_var_list + b_var_list, 'label' : dp_codes + b_codes}
codes_df = pd.DataFrame(data = codes_dict)
codes_df

,code,label
0,DP02_0122E,Estimate!!ANCESTRY!!Total population
1,DP02_0001E,Estimate!!HOUSEHOLDS BY TYPE!!Total households
2,DP02_0015E,Estimate!!HOUSEHOLDS BY TYPE!!Total households...
3,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
4,DP03_0005PE,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...
5,DP04_0001E,Estimate!!HOUSING OCCUPANCY!!Total housing units
6,DP04_0050E,Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Oc...
7,B01001_001E,Estimate!!Total
8,B01002_001E,Estimate!!Median age --!!Total
9,B06001_013E,Estimate!!Total!!Born in state of residence


In [9]:
# Request the Data Profile table
dp_call = requests.get(dp_url).json()
dp_df = pd.DataFrame(dp_call[1:len(dp_call)], columns = dp_call[0], dtype = float)

In [10]:
# Request the Detail table
b_call = requests.get(b_url).json()
b_df = pd.DataFrame(b_call[1:len(b_call)], columns = b_call[0], dtype = float)

In [11]:
# Merge the ACS tables
acs_df = dp_df.merge(b_df, left_on = ['NAME', 'state', 'county'], right_on = ['NAME', 'state', 'county'])

In [12]:
acs_df= acs_df.astype({'state': 'object', 'county':'object'})
acs_df['fips'] = acs_df['state']*1000 + acs_df['county']
acs_df

,NAME,DP02_0122E,DP02_0001E,DP02_0015E,DP03_0001E,DP03_0005PE,DP04_0001E,DP04_0050E,state,county,...,B06009_005E,B06009_006E,B06012_002E,B08133_001E,B19013_001E,B19301_001E,B19326_001E,B25071_001E,B25077_001E,fips
0,"Washington County, Mississippi",47086.0,18299.0,2.54,36109.0,8.4,21591.0,18299.0,28,151,...,3476.0,2220.0,15496.0,277765.0,30834.0,19884.0,19069.0,35.6,74700.0,28151
1,"Perry County, Mississippi",12028.0,4563.0,2.61,9508.0,3.5,5620.0,4563.0,28,111,...,694.0,204.0,2231.0,NaN,39007.0,21611.0,19724.0,29.0,83700.0,28111
2,"Choctaw County, Mississippi",8321.0,3164.0,2.58,6718.0,4.5,4200.0,3164.0,28,19,...,625.0,399.0,1888.0,NaN,37203.0,20589.0,18967.0,31.9,81600.0,28019
3,"Itawamba County, Mississippi",23480.0,8706.0,2.57,18949.0,2.2,10289.0,8706.0,28,57,...,1337.0,772.0,3428.0,228155.0,40510.0,20629.0,21698.0,27.4,89500.0,28057
4,"Carroll County, Mississippi",10129.0,3658.0,2.72,8404.0,4.5,5154.0,3658.0,28,15,...,692.0,359.0,1477.0,NaN,43060.0,22567.0,22091.0,36.0,89300.0,28015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,"Clayton County, Iowa",17672.0,7652.0,2.27,14243.0,1.4,9099.0,7652.0,19,43,...,1670.0,496.0,1564.0,186660.0,52828.0,28486.0,28407.0,23.5,121000.0,19043
3199,"Buena Vista County, Iowa",20260.0,7550.0,2.55,15640.0,3.8,8296.0,7550.0,19,21,...,1751.0,755.0,2507.0,136945.0,54556.0,26607.0,28553.0,23.9,112700.0,19021
3200,"Guthrie County, Iowa",10674.0,4397.0,2.39,8530.0,2.2,5786.0,4397.0,19,77,...,1093.0,393.0,1175.0,136575.0,57075.0,28953.0,30675.0,24.2,115900.0,19077
3201,"Humboldt County, Iowa",9566.0,4221.0,2.24,7572.0,2.4,4729.0,4221.0,19,91,...,1089.0,304.0,1103.0,83660.0,52219.0,29882.0,29958.0,25.1,102300.0,19091


In [13]:
acs_df["% Housing Units Occupied"] = acs_df["DP02_0001E"]/acs_df["DP04_0001E"]
acs_df["% Unemployment Rate"] = acs_df["DP03_0005PE"]/100
acs_df["% Born in State"] = acs_df["B06001_013E"]/acs_df["B01001_001E"]
acs_df["% <HS"] = acs_df["B06009_002E"]/acs_df["B01001_001E"]
acs_df["% HS Grad"] = acs_df["B06009_003E"]/acs_df["B01001_001E"]
acs_df["% Some College"] = acs_df["B06009_004E"]/acs_df["B01001_001E"]
acs_df["% Bach Degree"] = acs_df["B06009_005E"]/acs_df["B01001_001E"]
acs_df["% Grad Degree"] = acs_df["B06009_006E"]/acs_df["B01001_001E"]
acs_df["% Below Pov Level"] = acs_df["B06012_002E"]/acs_df["B01001_001E"]
acs_df["Commute Time"] = acs_df["B08133_001E"]
acs_df["Median Income"] = acs_df["B19326_001E"]
acs_df["Median Home Value"] = acs_df["B25077_001E"]

In [14]:
for col in acs_df.columns: 
    print(col)

NAME
DP02_0122E
DP02_0001E
DP02_0015E
DP03_0001E
DP03_0005PE
DP04_0001E
DP04_0050E
state
county
B01001_001E
B01002_001E
B06001_013E
B06009_002E
B06009_003E
B06009_004E
B06009_005E
B06009_006E
B06012_002E
B08133_001E
B19013_001E
B19301_001E
B19326_001E
B25071_001E
B25077_001E
fips
% Housing Units Occupied
% Unemployment Rate
% Born in State
% <HS
% HS Grad
% Some College
% Bach Degree
% Grad Degree
% Below Pov Level
Commute Time
Median Income
Median Home Value


In [15]:
model_df = acs_df[["NAME", "fips", "state", "% Housing Units Occupied", "% Unemployment Rate", "% Born in State", "% <HS", "% HS Grad",
                  "% Some College", "% Bach Degree", "% Grad Degree", "% Below Pov Level", "Commute Time", "Median Income", "Median Home Value"]]
for column in model_df.columns:
    print(f"Column {column} has {model_df[column].isnull().sum()} null values")

Column NAME has 0 null values
Column fips has 0 null values
Column state has 0 null values
Column % Housing Units Occupied has 62 null values
Column % Unemployment Rate has 0 null values
Column % Born in State has 62 null values
Column % <HS has 62 null values
Column % HS Grad has 62 null values
Column % Some College has 62 null values
Column % Bach Degree has 62 null values
Column % Grad Degree has 62 null values
Column % Below Pov Level has 63 null values
Column Commute Time has 547 null values
Column Median Income has 1 null values
Column Median Home Value has 0 null values


In [16]:
model_df = model_df.dropna()
model_df = pd.get_dummies(model_df, columns = ['state'])
model_df

,NAME,fips,% Housing Units Occupied,% Unemployment Rate,% Born in State,% <HS,% HS Grad,% Some College,% Bach Degree,% Grad Degree,...,state_46.0,state_47.0,state_48.0,state_49.0,state_50.0,state_51.0,state_53.0,state_54.0,state_55.0,state_56.0
0,"Washington County, Mississippi",28151,0.847529,0.084,0.868199,0.138534,0.196534,0.187635,0.073822,0.047148,...,0,0,0,0,0,0,0,0,0,0
3,"Itawamba County, Mississippi",28057,0.846146,0.022,0.752598,0.143739,0.202002,0.232411,0.056942,0.032879,...,0,0,0,0,0,0,0,0,0,0
9,"Quitman County, Mississippi",28119,0.849791,0.136,0.890667,0.196283,0.201980,0.194384,0.051275,0.024824,...,0,0,0,0,0,0,0,0,0,0
10,"Bolivar County, Mississippi",28011,0.846609,0.050,0.858708,0.147398,0.157738,0.181977,0.091372,0.060843,...,0,0,0,0,0,0,0,0,0,0
11,"Lamar County, Mississippi",28073,0.903702,0.062,0.691064,0.060827,0.154517,0.211244,0.146154,0.085817,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,"Clayton County, Iowa",19043,0.840972,0.014,0.735344,0.062981,0.311340,0.217746,0.094500,0.028067,...,0,0,0,0,0,0,0,0,0,0
3199,"Buena Vista County, Iowa",19021,0.910077,0.038,0.583860,0.140178,0.185686,0.180553,0.086426,0.037266,...,0,0,0,0,0,0,0,0,0,0
3200,"Guthrie County, Iowa",19077,0.759938,0.022,0.825745,0.050497,0.272063,0.243020,0.102398,0.036818,...,0,0,0,0,0,0,0,0,0,0
3201,"Humboldt County, Iowa",19091,0.892578,0.024,0.783713,0.054359,0.250993,0.247543,0.113841,0.031779,...,0,0,0,0,0,0,0,0,0,0


In [17]:
model_df.loc[model_df['fips'] == 6081]

,NAME,fips,% Housing Units Occupied,% Unemployment Rate,% Born in State,% <HS,% HS Grad,% Some College,% Bach Degree,% Grad Degree,...,state_46.0,state_47.0,state_48.0,state_49.0,state_50.0,state_51.0,state_53.0,state_54.0,state_55.0,state_56.0
1876,"San Mateo County, California",6081,0.947639,0.028,0.479206,0.077473,0.107286,0.173325,0.205704,0.151545,...,0,0,0,0,0,0,0,0,0,0


In [18]:
model_df.to_csv(r'counties.csv')

In [ ]:
model_df = model_df.drop(columns = ['NAME', 'fips'])

In [ ]:
# Split our preprocessed data into our features and target arrays
y = model_df["Median Home Value"]
X = model_df.drop(["Median Home Value"],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instance
#scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)
#y_scaler = scaler.fit(y_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)
#y_train_scaled = y_scaler.transform(y_train)
#y_test_scaled = y_scaler.transform(y_test)
X

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state=0)
regressor.fit(X_train, y_train)

In [ ]:
# Calculate the R^2 Score score
from sklearn.metrics import r2_score
y_pred = regressor.predict(X_test)
print("R^2 Score")
r2_score(y_test, y_pred)

In [ ]:
#Explained variance score
from sklearn.metrics import explained_variance_score
print("Explained variance")
explained_variance_score(y_test, y_pred)

In [ ]:
# List the features sorted in descending order by feature importance
importances = sorted(zip(regressor.feature_importances_, X.columns), reverse=True)
print("Top 10 Values ranked by importance")
importances[0:10]

In [ ]:
regressor.predict([model_df.drop(["Median Home Value"],1).loc[3202]])

In [ ]:
hypo = model_df.drop(["Median Home Value"],1).loc[3202]
hypo["% Housing Units Occupied"] = hypo["% Housing Units Occupied"]*1.1
print(hypo["% Housing Units Occupied"])
regressor.predict([hypo])


In [ ]:
#with open('housing_regressor.pkl', 'wb') as file:
#    pickle.dump(regressor, file)

In [ ]:
choro_df = acs_df[['fips', "NAME", "% Housing Units Occupied", "% Unemployment Rate", "% Born in State", "% <HS", "% HS Grad",
                  "% Some College", "% Bach Degree", "% Grad Degree", "% Below Pov Level", "Commute Time", "Median Income", "Median Home Value"]]
choro_df = choro_df.sort_values(by=['fips'])
choro_df['decile'] = pd.qcut(choro_df['Median Home Value'], 10, labels=False) + 1
#choro_df.to_csv("Resources/choro.csv", index= False)
choro_df = choro_df.set_index('fips')
choro_df.to_json("Resources/county.json", orient = 'index')
choro_df

In [ ]:
# counties = pd.read_json("https://raw.githubusercontent.com/python-visualization/folium/master/tests/us-counties.json", orient = "split)")
with open('Resources/us-counties.json', 'r') as f:
        counties = json.load(f)
#counties

In [ ]:
for feature in counties['features']:
    try:
        feature['properties']['home_value'] = int(choro_df.loc[float(feature['id']), "Median Home Value"])
        feature['properties']['decile'] = int(choro_df.loc[float(feature['id']), "decile"])
    except:
        feature['properties']['home_value'] = 'NA'
    #print(feature['id'], ":" ,feature['properties']['home_value'])
counties

In [ ]:
with open('Resources/choro.json', 'w') as f:
        json.dump(counties, f)